Spark LDA for NASA ADS v1.0 
8/2/2016
Tarun Ruchandani

To Do:
Generate document term matrix for LDA
Get LDA Results on small Corpus
Generate JS Matrix
Build Mapequation with JS Matrix
Extend to a larger corpus
Build Docker Container
Deploy to adsqb

In [28]:
#All 'em libraries

import numpy as np
import textmining
import pyspark
import lda
import lda.datasets
import ads
import pandas as pd
from textblob import TextBlob
from numpy  import array
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [29]:
# Extract Cosmology papers
ads.config.token = 'TkEI7jQxScyoCtzHfhcpCWVelRqySmH5XBksQCFA'
papers = list(ads.SearchQuery(q='cosmology'))

paper_abstracts=list()

for paper in papers:
    paper_abstracts.append(paper.abstract)

# Select first 25 papers which have abstracts. Some later ones don't. Revisit this when scaling.
paper_abstracts = paper_abstracts[0:25]


/Users/tarunruchandani/anaconda3/lib/python3.5/site-packages/ads/utils.py:23: UserWarning: You are lazy loading attributes via 'abstract', and so are making multiple calls to the API. This will impact your overall rate limits.
  UserWarning,


In [59]:
# Build Term-Document Matrix

tdm = textmining.TermDocumentMatrix()
for paper in paper_abstracts:
    tdm.add_doc(paper)
    
paper_abstracts_tdm = list()

for row in tdm.rows(cutoff=0):
    paper_abstracts_tdm.append(row)

paper_abstracts_tdm_df = pd.DataFrame(paper_abstracts_tdm)
paper_abstracts_tdm_df.head()

# Vocab of terms in abstracts
vocab = paper_abstracts_tdm_df._slice(slice(1))
vocab

d_t_freq = paper_abstracts_tdm_df._slice(slice(1,24))

b=d_t_freq.values

b=b.astype(int)

type(b)

# d_t_freq_m = d_t_freq.as_matrix
# d_t_freq_np = np.ndarray(b, dtype=np.int64)
# d_t_freq_np


numpy.ndarray

In [60]:
# running LDA on TDM

model = lda.LDA(n_topics=20, n_iter=500, random_state=1)
model.fit(b)

topic_word = model.topic_word_  # model.components_ also works
